# **Packages and Modules** 

Quando trabalhamos com testes unitarios muitas vezes estamos preocupados com o ***coverage***. Essa métrica mostra qual a porcentagem das linhas dos nossos arquivos que estão passando por testes.


 Suponha a seguinte estrutura de diretório:
```shell
.
├── src
│   ├── example
│   │   └── funcs.py
│   └── infra
│       ├── app.py
│       ├── model.py
│       └── processing.py
└── tests

```

Para ter o coverage desse código fonte normalmente executariamos o seguinte comando ```pytest tests/ --cov=src/```, onde ```--cov``` aceita diretórios ou nome de pacotes. Nessa situação o seguinte apareceria <font color ="red">**WARNING: Failed to generate report: No data to report.**</font>

Isso ocorre porque ```cov``` procurar por arquivos ```.py``` em um diretório ou pacote. Da forma como estava estruturado não existia arquivos no diretório.

Para resolver esse problema precisamos adicionar o arquivo ```__init__.py``` em todos os folders.
> The __init__.py files are required to make Python treat directories containing the file as packages - [link](https://docs.python.org/3/tutorial/modules.html). 

Agora quando passarmos ```cov=src``` estará sendo coberto todo o código ```src```.
```shell
.
├── src
│   ├── example
│   │   ├── funcs.py
│   │   └── __init__.py
│   ├── infra
│   │   ├── app.py
│   │   ├── __init__.py
│   │   ├── model.py
│   │   └── processing.py
│   └── __init__.py
└── tests

```

# **Possíveis Problemas**

## **Como mockar um import?** 

O python vai tentar importar e colocar o nome do pacote na variável ```sys.modules```

In [1]:
import sys
print("pandas" in sys.modules)

False


In [2]:
import pandas as pd
print("pd" in sys.modules)
print("pandas" in sys.modules)

False
True


In [3]:
# Como o python consegue encontrar o pacote é possível fazer as chamadas  
sys.modules["pandas"]

<module 'pandas' from '/home/rocabrera/.pyenv/versions/venv_playground/lib/python3.9/site-packages/pandas/__init__.py'>

In [4]:
df = pd.DataFrame([[1,2,3],[4,5,6]])
df

,0,1,2
0,1,2,3
1,4,5,6


O que acontece se a referência para o pacote fosse apagada?

In [5]:
sys.modules["pandas"] = None

In [6]:
df = pd.DataFrame([[1,2,3],[4,5,6]])

Com isso em mente, para mockar um import basta mudar a referência de import do pacote antes dos testes serem executados. Cria-se um arquivo denominado ```conftest.py```, o qual é executado pelo ***pytest*** antes dos test unitários, e adiciona-se o mock. 

Como exemplo:
```python
from unittest.mock import MagicMock 

sys.modules["pandas"] = MagicMock()  # Adiciona a chave pandas como um mock
```

A estrutura:
```shell
.
├── src
│   ├── example
│   │   ├── funcs.py
│   │   └── __init__.py
│   ├── infra
│   │   ├── app.py
│   │   ├── __init__.py
│   │   ├── model.py
│   │   └── processing.py
│   └── __init__.py
└── tests
    └── conftest.py
```

## Extra - Porque isso funciona? 

Isso funciona porque o python só importa um pacote que não está registrado.
Como exemplo: 

In [7]:
sys.modules["numpy"] = "batata"  # Adiciona a chave numpy
import numpy as np  # Import não tem efeito porque a variável ja existe
sys.modules["numpy"] 

'batata'

In [8]:
sys.modules.pop("numpy")  # Deleta o nome da variável
import numpy as np  # Se não existe numpy  o import funciona normalmente
sys.modules["numpy"]

/tmp/ipykernel_29033/2854398591.py:2: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy as np  # Se não existe numpy  o import funciona normalmente


<module 'numpy' from '/home/rocabrera/.pyenv/versions/venv_playground/lib/python3.9/site-packages/numpy/__init__.py'>

E esse é o motivo do porque precisamos restart o kernel para que as mudanças feitas no nosso arquivo reflitam ao realizar um novo import.